In [28]:
# !pip install finance-datareader
# !pip install pykrx

In [29]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
from pykrx import stock

In [30]:
today_date = '20230921'
stock_code = '005930' # 삼성전자

In [31]:
def todays_list(date):
    stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")}) # KOSPI, KOSDAQ, KONEX, ALL, (default=KOSPI)
    stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))
    stock_fud = pd.DataFrame(stock.get_market_fundamental_by_ticker(date=today_date, market="ALL"))
    stock_fud = stock_fud.reset_index()
    stock_fud.rename(columns={'티커':'종목코드'}, inplace=True)
    result = pd.merge(stock_list, stock_fud, left_on='종목코드', right_on='종목코드', how='outer')
    stock_price = stock.get_market_ohlcv_by_ticker(date=today_date, market="ALL")
    stock_price = stock_price.reset_index()
    stock_price.rename(columns={'티커':'종목코드'}, inplace=True)
    result1 = pd.merge(result, stock_price, left_on='종목코드', right_on='종목코드', how='outer')
    result1 = result1.replace([0], np.nan)
    result1 = result1.dropna(axis=0)
    result1['내재가치'] = (result1['BPS'] + (result1['EPS']) * 10) / 2
    result1['적정주가 가격']=result1['EPS']*result1['PER']
    result1['내재가치/종가'] = (result1['내재가치'] / result1['종가'])

    print("Complete")
    
    return result1

In [23]:
df = todays_list(today_date)
df

Complete


,종목코드,종목명,BPS,PER,PBR,EPS,DIV,DPS,시가,고가,저가,종가,거래량,거래대금,등락률,내재가치,적정주가 가격,내재가치/종가
1,095570,AJ네트웍스,8076.0,20.97,0.52,201.0,6.41,270.0,4365.0,4365.0,4205.0,4215,213127.0,9.146288e+08,-3.33,5043.0,4214.97,1.196441
4,265520,AP시스템,17980.0,3.26,0.99,5463.0,1.52,270.0,17830.0,18040.0,17750.0,17810,55527.0,9.901293e+08,-0.45,36305.0,17809.38,2.038462
5,211270,AP위성,5997.0,54.70,2.62,287.0,0.45,70.0,15680.0,16390.0,15380.0,15700,690592.0,1.099906e+10,-0.51,4433.5,15698.90,0.282389
6,027410,BGF,16528.0,14.53,0.22,247.0,3.06,110.0,3630.0,3630.0,3575.0,3590,38919.0,1.399728e+08,-1.10,9499.0,3588.91,2.645961
7,282330,BGF리테일,55724.0,12.89,2.59,11203.0,2.84,4100.0,147900.0,149000.0,144200.0,144400,28323.0,4.132667e+09,-2.83,83877.0,144406.67,0.580866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,243070,휴온스,23877.0,23.75,1.89,1897.0,1.33,600.0,44400.0,45250.0,43700.0,45050,54242.0,2.420919e+09,0.33,21423.5,45053.75,0.475549
2745,024060,흥구석유,5508.0,38.25,1.27,183.0,2.14,150.0,7320.0,7340.0,6830.0,7000,2057121.0,1.446012e+10,-8.02,3669.0,6999.75,0.524143
2746,010240,흥국,7971.0,8.62,0.80,740.0,3.45,220.0,6460.0,6470.0,6330.0,6380,9968.0,6.373477e+07,-1.85,7685.5,6378.80,1.204624
2747,189980,흥국에프엔비,2295.0,7.25,0.98,309.0,1.79,40.0,2280.0,2295.0,2200.0,2240,70025.0,1.561841e+08,-1.75,2692.5,2240.25,1.202009


In [25]:
df.loc[df['종가']<df['적정주가 가격']]

,종목코드,종목명,BPS,PER,PBR,EPS,DIV,DPS,시가,고가,저가,종가,거래량,거래대금,등락률,내재가치,적정주가 가격,내재가치/종가
7,282330,BGF리테일,55724.0,12.89,2.59,11203.0,2.84,4100.0,147900.0,149000.0,144200.0,144400,28323.0,4.132667e+09,-2.83,83877.0,144406.67,0.580866
8,126600,BGF에코머티리얼즈,6585.0,4.52,0.72,1053.0,2.10,100.0,4835.0,4865.0,4690.0,4755,131969.0,6.272107e+08,-2.16,8557.5,4759.56,1.799685
10,001460,BYC,825735.0,9.43,0.48,42272.0,0.75,3000.0,399500.0,408500.0,395500.0,398500,397.0,1.597985e+08,-0.25,624227.5,398624.96,1.566443
14,001040,CJ,191537.0,15.11,0.48,6023.0,2.75,2500.0,89200.0,93600.0,89200.0,91000,192614.0,1.767337e+10,1.00,125883.5,91007.53,1.383335
23,097950,CJ제일제당,459670.0,8.15,0.66,37188.0,1.82,5500.0,303000.0,308500.0,302500.0,303000,15935.0,4.856322e+09,-0.98,415775.0,303082.20,1.372195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717,004800,효성,128334.0,80.28,0.49,786.0,7.13,4500.0,63800.0,64300.0,63000.0,63100,15090.0,9.572178e+08,-1.87,68097.0,63100.08,1.079192
2738,065510,휴비츠,9709.0,10.94,1.81,1610.0,1.14,200.0,17690.0,17790.0,17190.0,17610,64604.0,1.125725e+09,-1.07,12904.5,17613.40,0.732794
2742,243070,휴온스,23877.0,23.75,1.89,1897.0,1.33,600.0,44400.0,45250.0,43700.0,45050,54242.0,2.420919e+09,0.33,21423.5,45053.75,0.475549
2747,189980,흥국에프엔비,2295.0,7.25,0.98,309.0,1.79,40.0,2280.0,2295.0,2200.0,2240,70025.0,1.561841e+08,-1.75,2692.5,2240.25,1.202009


In [8]:
from datetime import datetime

today_date = str(datetime.today()).split(" ")[0].replace("-",'')

'20230921'

In [ ]:
def todays_list(date):
    stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")}) # KOSPI, KOSDAQ, KONEX, ALL, (default=KOSPI)
    stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))
    stock_fud = pd.DataFrame(stock.get_market_fundamental_by_ticker(date=today_date, market="ALL"))
    stock_fud = stock_fud.reset_index()
    stock_fud.rename(columns={'티커':'종목코드'}, inplace=True)
    result = pd.merge(stock_list, stock_fud, left_on='종목코드', right_on='종목코드', how='outer')
    stock_price = stock.get_market_ohlcv_by_ticker(date=today_date, market="ALL")
    stock_price = stock_price.reset_index()
    stock_price.rename(columns={'티커':'종목코드'}, inplace=True)
    result1 = pd.merge(result, stock_price, left_on='종목코드', right_on='종목코드', how='outer')
    result1 = result1.replace([0], np.nan)
    result1 = result1.dropna(axis=0)
    result1['내재가치'] = (result1['BPS'] + (result1['EPS']) * 10) / 2
    result1['적정주가 가격']=result1['EPS']*result1['PER']
    result1['내재가치/종가'] = (result1['내재가치'] / result1['종가'])

    print("Complete")
    
    return result1

In [15]:
stock_list = pd.DataFrame({'005930':stock.get_market_ticker_list(market="ALL")})
stock_list

,005930
0,060310
1,095570
2,006840
3,054620
4,265520
...,...
2748,000540
2749,000545
2750,003280
2751,037440


In [17]:
name = stock.get_market_ticker_name('005930')

'삼성전자'

In [18]:
import requests, json, time
from bs4 import BeautifulSoup
from datetime import datetime

In [19]:
def get_price(company_code):
    bs_obj, url = get_code(company_code)
    no_today = bs_obj.find("p", {"class": "no_today"})
    blind = no_today.find("span", {"class": "blind"})
    now_price = blind.text
    return now_price, url
def get_code(company_code):
    url = "https://finance.naver.com/item/main.nhn?code=" + company_code
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return bs_obj, url

In [24]:
today_date = str(datetime.today()).split(" ")[0].replace("-",'')
code = '005930'

def stock_info(code):
    today_date = str(datetime.today()).split(" ")[0].replace("-",'')
    result = {}
    name = stock.get_market_ticker_name('005930')
    price, _ = get_price(code)
    
    result['종목명']=name
    result['현재가']=price
    return result

stock_info(code)

{'종목명': '삼성전자', '현재가': '68,900'}

In [33]:
df = todays_list(today_date).set_index('종목코드')
df

Complete


,종목명,BPS,PER,PBR,EPS,DIV,DPS,시가,고가,저가,종가,거래량,거래대금,등락률,내재가치,적정주가 가격,내재가치/종가
종목코드,,,,,,,,,,,,,,,,,
095570,AJ네트웍스,8076.0,21.00,0.52,201.0,6.40,270.0,4365.0,4365.0,4205.0,4220,243226.0,1.041793e+09,-3.21,5043.0,4221.00,1.195024
265520,AP시스템,17980.0,3.25,0.99,5463.0,1.52,270.0,17830.0,18040.0,17750.0,17760,63490.0,1.131742e+09,-0.73,36305.0,17754.75,2.044200
211270,AP위성,5997.0,54.95,2.63,287.0,0.44,70.0,15680.0,16390.0,15380.0,15770,1153052.0,1.839210e+10,-0.06,4433.5,15770.65,0.281135
027410,BGF,16528.0,14.53,0.22,247.0,3.06,110.0,3630.0,3630.0,3575.0,3590,47792.0,1.717714e+08,-1.10,9499.0,3588.91,2.645961
282330,BGF리테일,55724.0,12.87,2.59,11203.0,2.84,4100.0,147900.0,149000.0,144000.0,144200,38908.0,5.658791e+09,-2.96,83877.0,144182.61,0.581671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243070,휴온스,23877.0,23.46,1.86,1897.0,1.35,600.0,44400.0,45250.0,43700.0,44500,69695.0,3.111648e+09,-0.89,21423.5,44503.62,0.481427
024060,흥구석유,5508.0,38.42,1.28,183.0,2.13,150.0,7320.0,7340.0,6830.0,7030,2322904.0,1.631198e+10,-7.62,3669.0,7030.86,0.521906
010240,흥국,7971.0,8.58,0.80,740.0,3.46,220.0,6460.0,6470.0,6310.0,6350,15072.0,9.606231e+07,-2.31,7685.5,6349.20,1.210315


In [50]:
df_pivot = df[['종목명','종가','BPS','PER','PBR','EPS','DIV','DPS']]
check_lists=['BPS','PER']
value_rank = df_pivot[check_lists].rank(axis = 0) #열을 선택한 후 , 각열을 기준으로 순위를 구함
value_sum = value_rank.sum(axis = 1, skipna = False).rank() #더해진 값으로 랭킹을 다시 구함
value = df_pivot.loc[value_sum > value_sum.max()-20] # 밸류에이션이 낮은 20개 종목
value # 데이터 프레임 보여주기

,종목명,종가,BPS,PER,PBR,EPS,DIV,DPS
종목코드,,,,,,,,
007070,GS리테일,24000,39627.0,60.76,0.61,395.0,1.79,430.0
060980,HL홀딩스,35450,102608.0,112.90,0.35,314.0,5.64,2000.0
002380,KCC,257000,695902.0,56.31,0.37,4564.0,3.11,8000.0
051900,LG생활건강,448000,364213.0,31.75,1.23,14112.0,0.89,4000.0
051910,LG화학,525000,445524.0,22.18,1.18,23670.0,1.90,10000.0
035420,NAVER,210000,151646.0,41.94,1.38,5007.0,0.44,914.0
000660,SK하이닉스,116500,92004.0,35.93,1.27,3242.0,1.03,1200.0
004370,농심,435500,398464.0,21.69,1.09,20081.0,1.15,5000.0
000640,동아쏘시오홀딩스,96400,157100.0,54.04,0.61,1784.0,1.56,1500.0


In [51]:
len(value)

20

In [43]:
value

,종목명,종가,BPS,PER,PBR,EPS,DIV,DPS
종목코드,,,,,,,,
018500,동원금속,1777,1919.0,4.57,0.93,389.0,1.13,20.0
011370,서한,934,3928.0,2.39,0.24,390.0,5.35,50.0
035890,서희건설,1370,3361.0,3.02,0.41,454.0,3.28,45.0
036710,심텍홀딩스,2720,4971.0,3.83,0.55,711.0,1.84,50.0
037760,쎄니트,1517,2867.0,5.58,0.53,272.0,3.30,50.0
332370,아이디피,3505,3813.0,4.51,0.92,778.0,4.28,150.0
158430,아톤,3670,3159.0,4.52,1.16,812.0,0.54,20.0
031330,에스에이엠티,2795,3685.0,4.74,0.76,590.0,8.23,230.0
021080,에이티넘인베스트,2825,3553.0,4.71,0.80,600.0,5.31,150.0
